In [1245]:
import tkinter as tk
import webbrowser
from tkinter import ttk
from tkinter import filedialog
from PIL import ImageTk, Image
import string
import io
import csv
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [1246]:
# global var
W = np.array([[-1.234352  ],
       [-1.0965089 ],
       [ 0.9574655 ],
       [ 0.07142846],
       [ 0.18221012],
       [-1.110761  ],
       [ 0.49294937],
       [-0.5562565 ],
       [-0.27289093],
       [-0.3349833 ],
       [ 0.2631448 ],
       [-0.32123747],
       [ 1.0309917 ],
       [ 0.5341499 ],
       [-1.1661073 ],
       [-0.22655593],
       [-0.3114538 ],
       [ 0.50718987],
       [-0.9014772 ],
       [ 0.533391  ],
       [-0.1640348 ],
       [ 0.3487134 ],
       [-0.2405224 ],
       [ 0.54387933],
       [-0.12337092]])
b = np.array([[-0.99959964]])

wordsBag = ['likelyhood', 'breach', 'trade', 'evidence', 'property', 'public', 'opponents', 'constitution', 'fiduciary', 'confusion', 'crimes', 'copyright', 'intellectual', 'misuse', 'death', 'marks', 'constitutional', 'unregistered', 'contract', 'drugs', 'proprietor', 'similar', 'penalty', 'company', 'criminal']

countVec = []


Ww = np.array([[ 0.8594918 ],
       [ 0.8459306 ],
       [ 1.6249211 ],
       [ 1.8483686 ],
       [-0.36843148],
       [ 1.751518  ]])
bb = np.array([[-3.5971665]])

scoreVec = []

In [1247]:
def choose_file(event=None):
    f = filedialog.askopenfilename()
    fileName.set(f)
    fileNameShow.set(f[-9:])

In [1248]:
def open_browser():
    if len(fileName.get()) > 0:
        webbrowser.open('file://' + os.path.realpath(fileName.get()))

In [1249]:
def open_browser_action():
    if caseType.get() == 'Section 8(2)(b)':
        url = 'https://sso.agc.gov.sg/Act/TMA1998?ProvIds=P1II-#pr8-'
        webbrowser.open(url)

In [1250]:
def sigmoid(a):
    return 1/(1+np.exp(-a))

In [1251]:
def read_html(fileName):
    html = io.open(fileName.get(), mode="r", encoding="utf-8")
    soup = BeautifulSoup(html, 'html.parser') 
    return soup

In [1252]:
def read_stop_words(stopFileName):
    file = open(stopFileName, mode="r", encoding="utf-8")
    words = list(map(lambda x: x[0:-1], file.readlines()))
    return words

In [1253]:
def class_contents(soup, className):
    lst = soup.find_all(class_=className)
    text = '\n'.join([''.join(l.findAll(text=True)) + ' ' for l in lst])
    return text

In [1254]:
def clean_text(s, stopWords):
    puncs = string.punctuation.translate({ord('('): None, ord(')'): None}) + '’' + '“' + '”' + '\\'
    return list(map(lambda x: x.lower(), list(filter(lambda x:  4 < len(x) <= 16 and not any(p in x for p in puncs) and x not in stopWords and x.count('(') == x.count(')'), s.split(' ')))))

In [1255]:
def word_counter_vectorizer(text, wordsBag):
    vecLength = len(wordsBag)
    vec = [None] * vecLength
    for idx, word in enumerate(wordsBag):
        vec[idx] = text.count(word)
    return vec

In [1256]:
def classify_case():
    model = np.matmul(countVec, W) + b
    prediction = np.sign(model)
    global caseType
    if prediction >= 0:
        caseType.set('Section 8(2)(b)')
    else:
        caseType.set('None Trade Mark')

In [1257]:
def create_data(className='txt-body', stopWords='./stopWords.txt'):
    global countVec
    global caseSum
    if len(fileName.get()) > 0:
        soup = read_html(fileName)
        text = class_contents(soup, className)
        caseSum.set('\n'.join(list(filter(lambda x: len(x) < 50, text.split('\n')))[:]))
        cleanText = clean_text(text, read_stop_words(stopWords))
        countVec = word_counter_vectorizer(cleanText, wordsBag)
        countVec = np.array(countVec)
        countVec.astype(np.float64)
        classify_case()
    else:
        caseType.set('Choose a case first')  

In [1258]:
def similarity_score(t, score):
    def _record_score(v):
        score.set(v)
    s = tk.Scale(t, label='Score', from_=0, to=10, orient=tk.VERTICAL,
             length=300, showvalue=0, tickinterval=1, resolution=0.5, command=_record_score, bg = '#b3cde0', fg='#005b96',
            font=('Helvitica', 14))
    s.pack()
    l = tk.Label(t, 
        textvariable= score,
        bg ='#b3cde0',
        fg = '#d9534f',
        font=('Helvitica', 22),
        height=2)
    l.pack() 

In [1259]:
def new_window(): # new window definition
    def get_function(cmd):
        def change_text():
            if 'ms' == cmd:
                explaination.set('What is the similarity of marks as a whole?\nTaking a step back, how similar are the marks\nwhen considered as a whole?')
            if 'vs' == cmd:
                explaination.set('How similar do the marks look visually?\nThe more similar the dominant components, colours, shapes are,\nthe more similar the marks will look visually.')
            if 'as' == cmd:
                explaination.set('How similar or identical do the marks sound?\nConsider the number of common syllables.')
            if 'cs' == cmd:
                explaination.set('How similar is the underlying idea behind the mark?\nIf both marks point quite close to a specific idea,\nthey are similar conceptually.')
            if 'gs' == cmd:
                explaination.set('What is the similarity of goods & services (“G&S”)\nthat the marks are registered for?\nConsider the identity of the registration\nunder the respective classes of goods & services,\nthen the specific goods & services within the class.')
            if 'lc' == cmd:
                explaination.set('The more expensive the goods & services are,\nthe more popular the opposition mark,\nthe less likely there will be any confusion by the public.')
        return change_text
    explaination = tk.StringVar()
    newWin = tk.Toplevel(window)
    newWin.title('Trade Mark Cases')
    newWin.geometry('400x900')
    newWin.configure(background='#b3cde0')
    path = "./graph.png"
    img = ImageTk.PhotoImage(Image.open(path))
    lp = tk.Label(newWin, image=img, bg='#b3cde0')
    lp.pack() 
    l1 = tk.Label(newWin, text="3 Types Trade Mark Cases", fg = '#011f4b',
    bg='#b3cde0', font=('MS Sans Serif', 22))
    l1.pack()
    l2 = tk.Label(newWin, text="1. Text Logo\n2. Image Logo\n3. Text + Image Logo", fg = '#011f4b',
    bg='#b3cde0', font=('Helvitica', 20))
    l2.pack()
    l3 = tk.Label(newWin, 
        text='6 Principles:',
        fg = '#d9534f',
        bg='#b3cde0',
        font=('Helvitica', 22))
    l3.pack()
    b1 = tk.Button(newWin, 
        text ="1. Marks Similarity", 
        command =get_function('ms'), 
        font=('Helvitica', 18),
        width=18, height=2)
    b1.pack()
    b2 = tk.Button(newWin, 
        text ="(a) Visual Similarity", 
        command =get_function('vs'), 
        font=('Helvitica', 18),
        width=18, height=2)
    b2.pack()
    b3 = tk.Button(newWin, 
        text ="(b) Aural Similarity", 
        command =get_function('as'), 
        font=('Helvitica', 18),
        width=18, height=2)
    b3.pack()
    b4 = tk.Button(newWin, 
        text ="(c) Conceptual Similarity", 
        command =get_function('cs'), 
        font=('Helvitica', 18),
        width=18, height=2)
    b4.pack()
    b5 = tk.Button(newWin, 
        text ="2. G&S Similarity", 
        command =get_function('gs'), 
        font=('Helvitica', 18),
        width=18, height=2)
    b5.pack()
    b6 = tk.Button(newWin, 
        text ='3. Likelihood of Confusion', 
        command =get_function('lc'), 
        font=('Helvitica', 18),
        width=18, height=2)
    b6.pack()    
    b2 = tk.Button(newWin, 
        text ="Predict the Win Rate", 
        command =new_tab, 
        font=('Helvitica', 18),
        width=18, height=2)
    b2.pack()
    l4 = tk.Label(newWin, 
        textvariable=explaination,
        fg = '#d9534f',
        bg='#b3cde0',
        font=('Helvitica', 14),
        width = 40,
        height=5)
    l4.pack()
    newWin.mainloop()

In [1260]:
def new_tab():
    def _predict_result():
        global scoreVec
        global Ww
        global bb
        global winProb
        global winOrLose
        scoreVec = [score1.get(), score2.get(), score3.get(), score4.get(), score5.get(), score6.get()]
        scoreVec = np.asarray(scoreVec)
        scoreVec.astype(np.float64)
        model = np.matmul(scoreVec, Ww) + bb
        prediction = sigmoid(model)
        winProb.set(str(prediction[0][0] * 100)[0:4] + '%')
        if prediction[0][0] >= 0.7:
            winOrLose.set('Win')
        else:
            winOrLose.set('Lose')
        l1 = tk.Label(tab6, text=str(winProb.get()), bg='#b3cde0', fg='#d9534f',font=('Helvitica', 20))
        l1.pack()
        l2 = tk.Label(tab6, text=winOrLose.get(), bg='#b3cde0', fg='#d9534f', font=('Helvitica', 20))
        l2.pack()
        
    newTab = tk.Toplevel(window)
    newTab.title('Predict the Trade Mark Cases')
    newTab.geometry('400x900')
    # newTab.configure(background='#b3cde0')
    s = ttk.Style()
    s.configure('new.TFrame', background='#b3cde0')
    tabControl = ttk.Notebook(newTab)       
    tab1 = ttk.Frame(tabControl, style='new.TFrame')
    l1 = tk.Label(tab1, text="  Marks Similarity\n", anchor='w', bg ='#b3cde0', fg = '#011f4b', font=('Helvitica', 22))
    l1.pack(fill='both')
    path1 = "./tradeMark.png"
    img1 = ImageTk.PhotoImage(Image.open(path1))
    lp1 = tk.Label(tab1, image=img1, bg='#b3cde0')
    lp1.pack() 
    similarity_score(tab1, score1)
    tabControl.add(tab1, text='MS')
    # tab2
    tab2 = ttk.Frame(tabControl, style='new.TFrame')
    l2 = tk.Label(tab2, text="  Visual Similarity\n", anchor='w', bg ='#b3cde0', fg = '#011f4b', font=('Helvitica', 22))
    l2.pack(fill='both')
    path2 = "./attractive.png"
    img2 = ImageTk.PhotoImage(Image.open(path2))
    lp2 = tk.Label(tab2, image=img2, bg='#b3cde0')
    lp2.pack() 
    similarity_score(tab2, score2)
    tabControl.add(tab2, text='VS')
    # tab3
    tab3 = ttk.Frame(tabControl, style='new.TFrame')
    l3 = tk.Label(tab3, text="  Aural Similarity\n", anchor='w', bg ='#b3cde0', fg = '#011f4b', font=('Helvitica', 22))
    l3.pack(fill='both')
    path3 = "./ear.png"
    img3 = ImageTk.PhotoImage(Image.open(path3))
    lp3 = tk.Label(tab3, image=img3, bg='#b3cde0')
    lp3.pack() 
    similarity_score(tab3, score3)
    tabControl.add(tab3, text='AS')
    # tab4
    tab4 = ttk.Frame(tabControl, style='new.TFrame')
    l4 = tk.Label(tab4, text="  Conceptual Similarity\n", anchor='w', bg ='#b3cde0', fg = '#011f4b', font=('Helvitica', 22))
    l4.pack(fill='both')
    path4 = "./brainstorm.png"
    img4 = ImageTk.PhotoImage(Image.open(path4))
    lp4 = tk.Label(tab4, image=img4, bg='#b3cde0')
    lp4.pack()
    similarity_score(tab4, score4)
    tabControl.add(tab4, text='CS')
    # tab5
    tab5 = ttk.Frame(tabControl, style='new.TFrame')
    l5 = tk.Label(tab5, text="  G&S Similarity\n", anchor='w', bg ='#b3cde0', fg = '#011f4b', font=('Helvitica', 22))
    l5.pack(fill='both')
    path5 = "./groceries.png"
    img5 = ImageTk.PhotoImage(Image.open(path5))
    lp5 = tk.Label(tab5, image=img5, bg='#b3cde0')
    lp5.pack()
    similarity_score(tab5, score5)
    tabControl.add(tab5, text='GS')
    # tab6
    tab6 = ttk.Frame(tabControl, style='new.TFrame')
    l6 = tk.Label(tab6, text="  Likelihood of Confusion\n", anchor='w', bg ='#b3cde0', fg = '#011f4b', font=('Helvitica', 22))
    l6.pack(fill='both')
    path6 = "./psychology.png"
    img6 = ImageTk.PhotoImage(Image.open(path6))
    lp6 = tk.Label(tab6, image=img6, bg='#b3cde0')
    lp6.pack()
    similarity_score(tab6, score6)
    b = tk.Button(tab6, 
        text ="Predict the Win Rate", 
        command =_predict_result, 
        font=('Helvitica', 21),
        width = 14, height = 1)
    b.pack()
    tabControl.add(tab6, text='LC')
    tabControl.pack(expand=1, fill="both") 
    newTab.mainloop() 

In [1261]:
window = tk.Tk()
window.title('LIT')
window.geometry('420x900')
window.configure(background='#b3cde0')
fileName = tk.StringVar()
fileNameShow = tk.StringVar()
caseType = tk.StringVar()
caseSum = tk.StringVar()
score1 = tk.DoubleVar()
score2 = tk.DoubleVar()
score3 = tk.DoubleVar()
score4 = tk.DoubleVar()
score5 = tk.DoubleVar()
score6 = tk.DoubleVar()
winProb = tk.StringVar()
winOrLose = tk.StringVar()

In [1262]:
style = ttk.Style()

style.theme_create( "yummy", parent="alt", settings={
        "TNotebook": {"configure": {"tabmargins": [2, 5, 2, 0], "background": '#b3cde0'} },
        "TNotebook.Tab": {
            "configure": {"padding": [5, 1], "background": '#b3cde0' },
            "map":       {"background": [("selected", '#005b96')],
                          "expand": [("selected", [1, 1, 1, 0])] } } } )

style.theme_use("yummy")

In [1263]:
path = "./stadistics.png"
img = ImageTk.PhotoImage(Image.open(path))
lp = tk.Label(window, image=img, bg='#b3cde0')
lp.pack() 

In [1264]:
l0 = tk.Label(window, 
    text= 'Trade Mark Cases',
    fg = '#011f4b',
    bg='#b3cde0', 
    font=('MS Sans Serif', 32), 
    width=20)
l0.pack() 

In [1265]:
l1 = tk.Label(window, 
    textvariable= fileNameShow,
    fg = '#011f4b',
    bg='#b3cde0',  
    font=('Helvitica', 16), 
    width=50, height=2)
l1.pack() 

In [1266]:
b1 = tk.Button(window, 
    text='Choose a Case',
    command=choose_file,
    font=('Helvitica', 20),
    height = 2)
b1.pack()

In [1267]:
l2 = tk.Label(window, 
    textvariable=caseType,
    fg = '#d9534f',
    bg='#b3cde0',  
    font=('Helvitica', 20),
    width=24, height=2)
l2.pack()

In [1268]:
b2 = tk.Button(window, 
    text='Classify the Case', 
    command=create_data,
    font=('Helvitica', 20),
    height = 2)
b2.pack()

In [1269]:
l3 = tk.Label(window, 
    textvariable=caseSum,
    fg = '#011f4b',
    bg='#b3cde0',  
    font=('Helvitica', 14),
    width=200, height=10)
l3.pack()

In [1270]:
b3 = tk.Button(window, 
    text ="Read case", 
    command=open_browser,
    font=('Helvitica', 20),
    width=20, height=2)
b3.pack()

In [1271]:
b4 = tk.Button(window, 
    text ="Read more 8(2)(b)", 
    command=open_browser_action,
    font=('Helvitica', 20),
    width=20, height=2)
b4.pack()

In [1272]:
b5 = tk.Button(window, 
    text ="Assessments", 
    command=new_window,
    font=('Helvitica', 20),
    width=20, height=2)
b5.pack()

In [1273]:
window.mainloop()